In [ ]:
class node: #Class to track the properties of each node in the search space
  def __init__(self, state, parent, action, depth, n):
    self.state = state # a n*n 2D matrix - 0 indicates blank
    self.parent = parent # a node that generated the current node
    self.action = action # a char label that shows which action from the parent state was chosen 
    self.depth=depth #an integer
    self.blankPos = self.findBlank(n) #stores blank coordinates x,y
    
  def findBlank(self, n): #return position of blank tile
    for i in range(n):
      for j in range(n):
        if(self.state[i][j]==0):
          return i,j

In [ ]:
from queue import Queue
from math import factorial
from copy import deepcopy

class board: #class that tracks properties and search state space for a given puzzle configuration 
  def __init__(self, init_node_state, goal_state, n=3): # default n = 3 for 8-tile puzzle
    self.n=n # for dimension of the board
    initNode = node(init_node_state, None, '', 0, self.n) #initNode will be the root node
    self.currentNode=initNode # current Node that is being looked into
    self.goalState=goal_state #target state
    self.fringeQ = Queue(factorial(n*n)) #fringe list for generated nodes (but not expanded nodes)
    self.fringeQ.put(initNode)
    self.expandedList = [] #List for expanded states, so they won't be repeated

  def displayBoard(self, aNode: node): #print the current node state
    for i in range(self.n):
      for j in range(self.n):
        print(aNode.state[i][j], end = ' ')
      print()

  def tracePathFromRoot(self, aNode: node): #when we get a solution we print the path, recursively. Prints from parent node
    if aNode == None:
      return
    self.tracePathFromRoot(aNode.parent)
    print('At Level: ', aNode.depth, ", take action: ", aNode.action, ", to get: ")
    self.displayBoard(aNode)
  
  def tracePathFromNode(self, aNode: node, level): #Made especially for BDS, where we print backwards. hence forward pass is printed, then the depth is carried here as well
    #Printing the reverse direction
    if aNode.action == 'T':
      a = 'B'
    elif aNode.action == 'B':
      a = 'T'
    elif aNode.action == 'R':
      a = 'L'
    else:
      a = 'R'
    self.displayBoard(aNode) 
    if(aNode.parent!=None):
      print('At Level: ', level, ", take action: ", a, ", to get: ") 
    else:
      return
    self.tracePathFromNode(aNode.parent, level+1)
  

  def childGen(self, aNode: node, dirn, childX, childY): #Generating the child node given the move
    x = aNode.blankPos[0]
    y = aNode.blankPos[1]
    childState = deepcopy(aNode.state)
    childState[x][y], childState[childX][childY] = childState[childX][childY], childState[x][y] #swap the tiles
    if(childState not in self.expandedList): #make sure same state isn't repeated before adding to fringe list
      #Probably almost all of the time of the algos, BDS/BFS, is spent in checking this. HashTable provides a near constant time solution (couldn't implement it here though)
      child = node(childState, aNode, dirn, aNode.depth+1, self.n) 
      self.fringeQ.put(child)

  def move(self, aNode: node, dirn): #generate the 4 corresponding nodes from each state. order of direction shouldn't matter much
    x = aNode.blankPos[0]
    y = aNode.blankPos[1]
    if(dirn=='R'):
      childX = x
      childY = y+1
      if(childY<self.n):
        self.childGen(aNode, dirn, childX, childY)        
    elif(dirn=='T'):
      childX = x -1
      childY = y
      if(childX>=0):
        self.childGen(aNode, dirn, childX, childY)
    elif(dirn=='B'):
      childX = x+1
      childY = y
      if(childX<self.n):
        self.childGen(aNode, dirn, childX, childY)
    else:
      childX = x
      childY = y-1
      if(childY>=0):
        self.childGen(aNode, dirn, childX, childY)

In [ ]:
def successorFnBFS(puzzle: board): #Plain BFS successor function
  status = ' '
  while not puzzle.fringeQ.empty(): 
    puzzle.currentNode = puzzle.fringeQ.get() #dequeue
    if(puzzle.currentNode.state == puzzle.goalState): #test
      puzzle.tracePathFromRoot(puzzle.currentNode) #just stop all node searching/generation and print the path
      status = 'Solved'
      return status
    #actual node generation through the 4 moves of the block
    for dirn in ['R', 'T', 'B', 'L']: 
      puzzle.move(puzzle.currentNode, dirn)
    puzzle.expandedList.append(puzzle.currentNode.state) #finally, as all the moves from the current node have been enumerated, archive the node

  if(status!='Solved'): #if fringe list is empty, it means all possible states were already explored, so no solution exists 
    status = 'Unsolvable'
  return status

In [ ]:
def BFSExpansion(puzzle): #similar to BFS Successor Function, but the looping is being taken care of the overall BDS fn below
  puzzle.currentNode = puzzle.fringeQ.get()
  if(puzzle.currentNode.state == puzzle.goalState):
      puzzle.tracePathFromRoot(puzzle.currentNode)
  for dirn in ['R', 'T', 'B', 'L']:
      puzzle.move(puzzle.currentNode, dirn)
  puzzle.expandedList.append(puzzle.currentNode.state)
  #keeping this seperate helps to control the expansion one node at a time, without having to modify the class method code to search the node in another tree's fringe list

def successorFnBDS(fwdPuzzle: board, bckPuzzle: board): #BDS successor function
  meet = 0 #track when the 2 trees finally meet
  
  while (not fwdPuzzle.fringeQ.empty()) and (not bckPuzzle.fringeQ.empty()) and meet!=1:
    for f in bckPuzzle.fringeQ.queue: #before dequeuing and expanding, see if the state about to be dequeued is already in the fringe list of the backward pass
      if(fwdPuzzle.fringeQ.queue[0].state == f.state): #again, could have searched through hashing but not as necessary as the one in childGen()
        meet = 1
        fwdPuzzle.tracePathFromRoot(fwdPuzzle.fringeQ.queue[0]) #if it meets, print from root to common node...
        print('---Meeting point---')
        bckPuzzle.tracePathFromNode(f, f.depth+1) #...and then from common node to goal
    if(meet==0): #if not meet, expand as usual, one node at a time
      BFSExpansion(fwdPuzzle)
      BFSExpansion(bckPuzzle)

  return meet

In [ ]:
#Driver code

initialState1 = [[1,2,3],
                [4,5,6],
                [7,0,8]]
goalState1 = [[1,2,3],
             [4,5,0],
             [7,8,6]]

#Let's see how  BFS and BDS do on a simple first puzzle
print("BFS (forward) solution")
forwardPuzzle = board(initialState1, goalState1)
forwardSoln = successorFnBFS(forwardPuzzle)
print(forwardSoln)
print()
print("BFS (backward) solution") #Sanity check
backwardPuzzle = board(goalState1, initialState1)
backwardSoln = successorFnBFS(backwardPuzzle)
print(backwardSoln)
print()
print("BDS Solution")
solnBDS = successorFnBDS(forwardPuzzle, backwardPuzzle)
if(solnBDS):
  print("Solved through BDS")
else:
  print("Unsolvable")

BFS (forward) solution
At Level:  0 , take action:   , to get: 
1 2 3 
4 5 6 
7 0 8 
At Level:  1 , take action:  R , to get: 
1 2 3 
4 5 6 
7 8 0 
At Level:  2 , take action:  T , to get: 
1 2 3 
4 5 0 
7 8 6 
Solved

BFS (backward) solution
At Level:  0 , take action:   , to get: 
1 2 3 
4 5 0 
7 8 6 
At Level:  1 , take action:  B , to get: 
1 2 3 
4 5 6 
7 8 0 
At Level:  2 , take action:  L , to get: 
1 2 3 
4 5 6 
7 0 8 
Solved

BDS Solution
At Level:  0 , take action:   , to get: 
1 2 3 
4 5 6 
7 0 8 
At Level:  1 , take action:  T , to get: 
1 2 3 
4 0 6 
7 5 8 
At Level:  2 , take action:  R , to get: 
1 2 3 
4 6 0 
7 5 8 
At Level:  3 , take action:  B , to get: 
1 2 3 
4 6 8 
7 5 0 
At Level:  4 , take action:  L , to get: 
1 2 3 
4 6 8 
7 0 5 
At Level:  5 , take action:  T , to get: 
1 2 3 
4 0 8 
7 6 5 
---Meeting point---
1 2 3 
4 0 8 
7 6 5 
At Level:  6 , take action:  R , to get: 
1 2 3 
4 8 0 
7 6 5 
At Level:  7 , take action:  B , to get: 
1 2 3 
4 8 5 
7 6 0 
At L

In [ ]:
#Now let's illustrate with a second example of why you should use BDS for more complicated trees
# initialState = [[6,0,3],
#                  [8,5,4],
#                  [7,2,1]]

initialState = [[8,1,2],
                 [0,4,3],
                 [7,6,5]]

goalState = [[1,2,3],
              [4,5,6],
              [8,7,0]]
forwardPuzzle = board(initialState, goalState)
backwardPuzzle = board(goalState, initialState)
print("BDS Solution")
solnBDS = successorFnBDS(forwardPuzzle, backwardPuzzle)
if(solnBDS):
  print("Solved through BDS")
else:
  print("Unsolvable through BDS")

BDS Solution
At Level:  0 , take action:   , to get: 
8 1 2 
0 4 3 
7 6 5 
At Level:  1 , take action:  T , to get: 
0 1 2 
8 4 3 
7 6 5 
At Level:  2 , take action:  R , to get: 
1 0 2 
8 4 3 
7 6 5 
At Level:  3 , take action:  R , to get: 
1 2 0 
8 4 3 
7 6 5 
At Level:  4 , take action:  B , to get: 
1 2 3 
8 4 0 
7 6 5 
At Level:  5 , take action:  B , to get: 
1 2 3 
8 4 5 
7 6 0 
---Meeting point---
1 2 3 
8 4 5 
7 6 0 
At Level:  7 , take action:  L , to get: 
1 2 3 
8 4 5 
7 0 6 
At Level:  8 , take action:  L , to get: 
1 2 3 
8 4 5 
0 7 6 
At Level:  9 , take action:  T , to get: 
1 2 3 
0 4 5 
8 7 6 
At Level:  10 , take action:  R , to get: 
1 2 3 
4 0 5 
8 7 6 
At Level:  11 , take action:  R , to get: 
1 2 3 
4 5 0 
8 7 6 
At Level:  12 , take action:  B , to get: 
1 2 3 
4 5 6 
8 7 0 
Solved through BDS


In [ ]:
print("BFS solution")
bfsSoln = successorFnBFS(forwardPuzzle) #I stopped at 26 minutes of run time without a solution. Compare this to the less-than-a-second solution by BDS!! 
print(bfsSoln)
#Note the first example: even though the BDS solution wasn't optimal and involved much more steps, it atleast gave the answer ~immediately. BFS gave neither here.

BFS solution


KeyboardInterrupt: ignored

In [ ]:
#That said, check this 3rd example out, where an innocent number swap difference in the initial and goal states causes even BDS to not give a solution appropriately...
initialState = [[1,2,3],
                [4,5,6],
                [8,7,0]]
goalState = [[1,2,3],
             [4,5,6],
             [7,8,0]]
forwardPuzzle = board(initialState, goalState)
backwardPuzzle = board(goalState, initialState)
solnBDS = successorFnBDS(forwardPuzzle, backwardPuzzle)
#... a stopped the run in 26 minutes again, without a solution

KeyboardInterrupt: ignored

In [ ]:
# def recursiveDLS(aNode: node, puzzle: board, limit):
#   cutoffOccurred = 0
#   cutoff=30
#   if(puzzle.currentNode.state == puzzle.goalState): #test
#     puzzle.tracePathFromRoot(puzzle.currentNode) #just stop all node searching/generation and print the path
#     return 'Success'
#   elif aNode.depth == limit:
#     return cutoff
#   else:
#     puzzle.currentNode = puzzle.fringeQ.get()
#     for dirn in ['R', 'T', 'B', 'L']: 
#       puzzle.move(puzzle.currentNode, dirn)
#     puzzle.expandedList.append(puzzle.currentNode.state) 
# # 
#     i=0
#     while i<len(puzzle.fringeQ) and not puzzle.fringeQ.empty():
#       result = recursiveDLS(puzzle.fringeQ.queue[i], puzzle, limit)
#       if result == cutoff:
#         cutoffOccurred = 1
#       elif result != 'Failure':
#         return result
    
#   if cutoffOccurred ==1:
#     return cutoff
#   else:
#     return 'Failure'


# def successorFnDLS(puzzle: board, limit):
#   return recursiveDLS(puzzle.currentNode,puzzle, limit)


# def successorFnIDS(puzzle: board): #IDS successor function
#   status = ' '
#   depth = 0
#   cutoff = 30
#   while status!='Solved':
#     result = successorFnDLS(puzzle, depth)
#     if result!=cutoff or result == 'Success':
#       status = 'Solved'
  
#   return status

#   while not puzzle.fringeQ.empty(): 
#     puzzle.currentNode = puzzle.fringeQ.get() #dequeue
#     if(puzzle.currentNode.state == puzzle.goalState): #test
#       puzzle.tracePathFromRoot(puzzle.currentNode) #just stop all node searching/generation and print the path
#       status = 'Solved'
#       return status
#     #actual node generation through the 4 moves of the block
#     for dirn in ['R', 'T', 'B', 'L']: 
#       puzzle.move(puzzle.currentNode, dirn)
#     puzzle.expandedList.append(puzzle.currentNode.state) #finally, as all the moves from the current node have been enumerated, archive the node

#   if(status!='Solved'): #if fringe list is empty, it means all possible states were already explored, so no solution exists 
#     status = 'Unsolvable'
#     return status